<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0728/09_1_(practice)FCN_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FCN_implementation

## 주요 함수
- x32, x16, x8 : tensorflow.keras.layers.Conv2DTranspose
    - Conv2DTranspose(filters, size, strides, padding='same')
- x2 : tensorflow.keras.layers.Upsampling2D
    - UpSampling2D(size, interpolation='bilinear')

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [35]:
def conv_block(x, filters, kernel_size):
    x = tf.keras.layers.Conv2D(filters , kernel_size,  padding = 'same', activation='relu')(x)
    return x

In [129]:
def FCN(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # VGG backborn
    x = conv_block(inputs, 64, 3)
    x = conv_block(x, 64, 3)
    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_block(x, 128, 3)
    x = conv_block(x, 128, 3)
    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_block(x, 256, 3)
    x = conv_block(x, 256, 3)
    x = conv_block(x, 256, 3)
    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    add1 = x

    x = conv_block(x, 512, 3)
    x = conv_block(x, 512, 3)
    x = conv_block(x, 512, 3)
    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    add2 = x

    x = conv_block(x, 512, 3)
    x = conv_block(x, 512, 3)
    x = conv_block(x, 512, 3)
    x = tf.keras.layers.MaxPooling2D(pool_size = 2, strides = 2)(x)

    x = conv_block(x, 4096, 1)
    x = conv_block(x, 4096, 1)

    x = conv_block(x, 6, 1)
    a1 = tf.keras.layers.UpSampling2D(32)(x)

    x = tf.keras.layers.Conv2DTranspose(512, kernel_size= 4,  strides = 2, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.Add()([add2, x])

    a2 = tf.keras.layers.UpSampling2D(16)(x)

    x = tf.keras.layers.Conv2DTranspose(256,kernel_size = 4, strides = 2, padding = 'same', activation = 'relu')(x)
    x = tf.keras.layers.Add()([add1, x])

    a3 = tf.keras.layers.UpSampling2D(8)(x)
    x = tf.keras.layers.Conv2DTranspose(256,kernel_size = 4, strides = 2, padding = 'same', activation = 'softmax')(a3)

    model = tf.keras.Model(inputs = inputs, outputs = [x, a1, a2, a3])
    return model

In [130]:
model = FCN(input_shape = [224,224,3])
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_53 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_689 (Conv2D)            (None, 224, 224, 64  1792        ['input_53[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_690 (Conv2D)            (None, 224, 224, 64  36928       ['conv2d_689[0][0]']             
                                )                                                          